## Roots Classical Music Project

### RagaVerse


In [3]:
import os
import numpy as np
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import get_window
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE


In [4]:
def audio_to_sine_wave(filepath, sr=16000, win_size=0.01):
    y, sr = librosa.load(filepath, sr=sr)
    f0, voiced_flag, _ = librosa.pyin(y, fmin=librosa.note_to_hz('C2'),
                                      fmax=librosa.note_to_hz('C7'),
                                      frame_length=int(win_size*sr))
    # Replace NaNs with 0s or interpolate
    f0 = np.nan_to_num(f0)
    t = np.linspace(0, len(y) / sr, len(f0))
    sine = np.sin(2 * np.pi * f0 * t)
    return f0, sine, sr


In [5]:
def hz_to_cents(f0, tonic=None):
    if tonic is None:
        # Use median non-zero value as tonic estimate
        tonic = np.median(f0[f0 > 0])
    cents = 1200 * np.log2(f0 / tonic)
    cents[f0 <= 0] = np.nan  # Ignore unvoiced
    return cents, tonic


In [6]:
def plot_pitch_contour(cents, sr, hop_length=512):
    time = np.arange(len(cents)) * hop_length / sr
    plt.figure(figsize=(10, 3))
    plt.plot(time, cents, label='Pitch contour (cents)')
    plt.xlabel('Time (s)')
    plt.ylabel('Pitch (cents)')
    plt.title('Pitch Contour')
    plt.grid(True)
    plt.show()


In [7]:
def compute_pitch_class_distribution(cents, bins=1200):
    # Wrap around 1200 cents (1 octave)
    pitch_classes = np.mod(cents, 1200)
    pitch_classes = pitch_classes[~np.isnan(pitch_classes)]
    hist, _ = np.histogram(pitch_classes, bins=bins, range=(0, 1200), density=True)
    return hist


In [8]:
def plot_2d_embedding(features, labels=None, method='pca'):
    if method == 'pca':
        reducer = PCA(n_components=2)
    elif method == 'tsne':
        reducer = TSNE(n_components=2, perplexity=5)
    else:
        raise ValueError("Choose 'pca' or 'tsne'")
    
    scaled = StandardScaler().fit_transform(features)
    embedding = reducer.fit_transform(scaled)
    
    plt.figure(figsize=(6, 5))
    sns.scatterplot(x=embedding[:, 0], y=embedding[:, 1], hue=labels)
    plt.title(f'{method.upper()} Projection of Pitch Class Distributions')
    plt.show()
